# Ppk vs % OOS Analysis for One-Sided Normal Distribution

This notebook demonstrates the relationship between Process Performance Index (Ppk) and the percentage of Out-of-Specification (OOS) units for a one-sided normal distribution.

## Theory

For a one-sided specification with an upper specification limit (USL):
- **Ppk = (USL - μ) / (3σ)** where μ is the process mean and σ is the process standard deviation
- **% OOS = 100 × Φ((μ - USL)/σ)** where Φ is the standard normal CDF

The relationship can be derived as:
- **% OOS = 100 × Φ(-3 × Ppk)**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd

# Set up plotting parameters for better visualization
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

In [ ]:
def ppk_to_oos_percent(ppk):
    """
    Convert Ppk to percentage Out-of-Specification for one-sided normal distribution.
    
    Parameters:
    ppk (float or array): Process Performance Index
    
    Returns:
    float or array: Percentage of units Out-of-Specification
    
    Formula: % OOS = 100 × Φ(-3 × Ppk)
    where Φ is the standard normal cumulative distribution function
    """
    return 100 * stats.norm.cdf(-3 * ppk)

def oos_percent_to_ppk(oos_percent):
    """
    Convert percentage Out-of-Specification to Ppk for one-sided normal distribution.
    
    Parameters:
    oos_percent (float or array): Percentage of units Out-of-Specification
    
    Returns:
    float or array: Process Performance Index
    
    Formula: Ppk = -Φ⁻¹(% OOS / 100) / 3
    where Φ⁻¹ is the inverse standard normal cumulative distribution function
    """
    return -stats.norm.ppf(oos_percent / 100) / 3

In [ ]:
# Generate Ppk values from 0.1 to 2.0
ppk_values = np.linspace(0.1, 2.0, 200)

# Calculate corresponding % OOS values
oos_percent_values = ppk_to_oos_percent(ppk_values)

# Create the main plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Ppk vs % OOS (linear scale)
ax1.plot(ppk_values, oos_percent_values, 'b-', linewidth=2, label='Ppk vs % OOS')
ax1.set_xlabel('Ppk (Process Performance Index)')
ax1.set_ylabel('% Out-of-Specification')
ax1.set_title('Ppk vs % OOS (Linear Scale)')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0.1, 2.0)

# Add secondary x-axis for Sigma levels
ax1_sigma = ax1.twiny()
ax1_sigma.set_xlim(0.3, 6.0)  # Sigma levels corresponding to Ppk 0.1-2.0
ax1_sigma.set_xlabel('Sigma Level (σ)', color='purple')
ax1_sigma.tick_params(axis='x', labelcolor='purple')

# Add some reference lines for common Ppk values (including 1-sigma)
reference_ppk = [0.33, 0.67, 1.0, 1.33, 1.67]  # Added 0.33 for 1-sigma
for ppk_ref in reference_ppk:
    oos_ref = ppk_to_oos_percent(ppk_ref)
    sigma_level = ppk_ref * 3  # Convert Ppk to sigma level
    ax1.axvline(x=ppk_ref, color='red', linestyle='--', alpha=0.5)
    ax1.text(ppk_ref, ax1.get_ylim()[1]*0.9, f'Ppk={ppk_ref}\n{sigma_level:.1f}σ\n{oos_ref:.3f}%', 
             ha='center', va='top', fontsize=9, 
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

# Plot 2: Ppk vs % OOS (log scale for % OOS)
ax2.semilogy(ppk_values, oos_percent_values, 'g-', linewidth=2, label='Ppk vs % OOS (log scale)')
ax2.set_xlabel('Ppk (Process Performance Index)')
ax2.set_ylabel('% Out-of-Specification (log scale)')
ax2.set_title('Ppk vs % OOS (Log Scale for % OOS)')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0.1, 2.0)

# Add secondary x-axis for Sigma levels on log plot
ax2_sigma = ax2.twiny()
ax2_sigma.set_xlim(0.3, 6.0)  # Sigma levels corresponding to Ppk 0.1-2.0
ax2_sigma.set_xlabel('Sigma Level (σ)', color='purple')
ax2_sigma.tick_params(axis='x', labelcolor='purple')

# Add reference lines for log plot
for ppk_ref in reference_ppk:
    oos_ref = ppk_to_oos_percent(ppk_ref)
    sigma_level = ppk_ref * 3  # Convert Ppk to sigma level
    ax2.axvline(x=ppk_ref, color='red', linestyle='--', alpha=0.5)
    ax2.text(ppk_ref, oos_ref*10, f'Ppk={ppk_ref}\n{sigma_level:.1f}σ\n{oos_ref:.2e}%', 
             ha='center', va='bottom', fontsize=9,
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

In [ ]:
# Create a summary table for common Ppk values
ppk_common = np.array([0.33, 0.67, 1.0, 1.33, 1.67, 2.0])
oos_common = ppk_to_oos_percent(ppk_common)

# Calculate defects per million opportunities (DPMO)
dpmo = oos_common * 10000  # Convert % to DPMO

summary_df = pd.DataFrame({
    'Ppk': ppk_common,
    '% OOS': oos_common,
    'DPMO': dpmo,
    'Sigma Level (approx)': ppk_common * 3  # Approximate sigma level
})

print("Summary Table: Ppk vs % OOS vs DPMO")
print("=" * 50)
print(summary_df.round(6))

Summary Table: Ppk vs % OOS vs DPMO
    Ppk      % OOS           DPMO  Sigma Level (approx)
0  0.33  16.108706  161087.059511                  0.99
1  0.67   2.221559   22215.594429                  2.01
2  1.00   0.134990    1349.898032                  3.00
3  1.33   0.003304      33.036648                  3.99
4  1.67   0.000027       0.272150                  5.01
5  2.00   0.000000       0.000987                  6.00


In [ ]:
# Interactive exploration: Calculate % OOS for user-defined Ppk
print("Interactive Calculation Examples:")
print("=" * 40)

# Example calculations
test_ppk_values = [0.5, 1.0, 1.5, 2.0]

for ppk in test_ppk_values:
    oos = ppk_to_oos_percent(ppk)
    print(f"Ppk = {ppk:4.1f} → % OOS = {oos:10.6f}% → DPMO = {oos*10000:10.2f}")

print("\nReverse calculation (% OOS to Ppk):")
print("-" * 40)

test_oos_values = [0.1, 1.0, 5.0, 10.0]

for oos in test_oos_values:
    ppk = oos_percent_to_ppk(oos)
    print(f"% OOS = {oos:5.1f}% → Ppk = {ppk:6.3f}")

Interactive Calculation Examples:
Ppk =  0.5 → % OOS =   6.680720% → DPMO =   66807.20
Ppk =  1.0 → % OOS =   0.134990% → DPMO =    1349.90
Ppk =  1.5 → % OOS =   0.000340% → DPMO =       3.40
Ppk =  2.0 → % OOS =   0.000000% → DPMO =       0.00

Reverse calculation (% OOS to Ppk):
----------------------------------------
% OOS =   0.1% → Ppk =  1.030
% OOS =   1.0% → Ppk =  0.775
% OOS =   5.0% → Ppk =  0.548
% OOS =  10.0% → Ppk =  0.427


## Key Insights

1. **Exponential Relationship**: The relationship between Ppk and % OOS is exponential - small improvements in Ppk lead to dramatic reductions in % OOS.

2. **Industry Benchmarks**:
   - **Ppk = 1.0**: Approximately 0.135% OOS (1,350 DPMO)
   - **Ppk = 1.33**: Approximately 0.0063% OOS (63 DPMO) - Often considered "adequate"
   - **Ppk = 1.67**: Approximately 0.000057% OOS (0.57 DPMO) - World class performance

3. **Practical Application**: Use this relationship to:
   - Set realistic quality targets
   - Estimate defect rates from Ppk measurements
   - Determine required process improvement for quality goals

## Mathematical Foundation

The mathematical relationship is based on the standard normal distribution:
- For a process with mean μ and standard deviation σ
- With upper specification limit USL
- The probability of exceeding USL is Φ((μ - USL)/σ)
- Since Ppk = (USL - μ)/(3σ), we get % OOS = 100 × Φ(-3 × Ppk)